Behavioral Cloning P3
--

In [1]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Cropping2D
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.models import load_model
import csv
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn
from random import shuffle

Using TensorFlow backend.


In [15]:
angle_adjust = [0, 0.2, -0.2]
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                # center, left, right
                for i in range(3):
                    name = './IMG/' + batch_sample[i].split('/')[-1]
                    image = cv2.imread(name)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    angle = float(batch_sample[3]) + angle_adjust[i]
                    images.append(image)
                    angles.append(angle)
                    # flip image and angle
                    image_flipped = np.fliplr(image)
                    angle_flipped = -angle
                    images.append(image_flipped)
                    angles.append(angle_flipped)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

In [22]:
samples = []
with open('./driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

BATCH_SIZE = 8
NB_EPOCH = 10
nb_filters_conv1 = 32
nb_filters_conv2 = 64
kernel_size = (3, 3)
pool_size = (2, 2)

train_samples, validation_samples = train_test_split(samples, test_size=0.2)
train_generator = generator(train_samples, batch_size=BATCH_SIZE)
validation_generator = generator(validation_samples, batch_size=BATCH_SIZE)

In [23]:
model = Sequential()
# preprocessing
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))
model.add(Lambda(lambda x: x / 255, input_shape=(160,320,3)))

# conv1
model.add(Convolution2D(32, kernel_size[0], kernel_size[1], border_mode='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

# conv2
model.add(Convolution2D(32, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

# conv3
model.add(Convolution2D(64, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.5))
model.add(Flatten())

# FC1
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Output
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
history = model.fit_generator(train_generator,
        samples_per_epoch=len(train_samples),
        validation_data=validation_generator,
        nb_val_samples=len(validation_samples),
        nb_epoch=NB_EPOCH)
model.save('model.h5')
print('model saved.')

Epoch 1/10
1344/1360 [============================>.] - ETA: 0s - loss: 0.2718

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


1392/1360 [==============================] - 10s - loss: 0.2633 - val_loss: 0.0346
Epoch 2/10
1392/1360 [==============================] - 10s - loss: 0.0269 - val_loss: 0.0155
Epoch 3/10
1392/1360 [==============================] - 10s - loss: 0.0229 - val_loss: 0.0094
Epoch 4/10
1392/1360 [==============================] - 10s - loss: 0.0181 - val_loss: 0.0081
Epoch 5/10
1392/1360 [==============================] - 10s - loss: 0.0144 - val_loss: 0.0087
Epoch 6/10
1392/1360 [==============================] - 10s - loss: 0.0199 - val_loss: 0.0047
Epoch 7/10
1392/1360 [==============================] - 10s - loss: 0.0136 - val_loss: 0.0091
Epoch 8/10
1392/1360 [==============================] - 10s - loss: 0.0124 - val_loss: 0.0061
Epoch 9/10
1392/1360 [==============================] - 10s - loss: 0.0149 - val_loss: 0.0095
Epoch 10/10
1392/1360 [==============================] - 10s - loss: 0.0128 - val_loss: 0.0073
model saved.


In [19]:
print(model.summary())
print(history.history)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_6 (Lambda)                (None, 160, 320, 3)   0           lambda_input_6[0][0]             
____________________________________________________________________________________________________
convolution2d_16 (Convolution2D) (None, 158, 318, 32)  896         lambda_6[0][0]                   
____________________________________________________________________________________________________
activation_21 (Activation)       (None, 158, 318, 32)  0           convolution2d_16[0][0]           
____________________________________________________________________________________________________
maxpooling2d_16 (MaxPooling2D)   (None, 79, 159, 32)   0           activation_21[0][0]              
___________________________________________________________________________________________

In [ ]:
model = Sequential()
model = load_model('model.h5')